# Feature Creation

This page will focus on creating features from cleaned-text data by counting word frequency and some limitations regarding using such a method. Also, we will talk about how we should handle non-text columns in the data, especially the columns with categorical variables. 

First, we will load the data we cleaned on the previous page, and we will split this into train and test datasets. We split the dataset to ensure the test dataset does not influence when creating a pipeline. Technically, we are not supposed to know our test set before the completion of our pipeline. **After we create the pipeline, we will apply the same pipeline we used for the train set to the test set to ensure both datasets have the same dimension.**  Please refer to [here](Plan.ipynb) if you want to know how I splited the dataset into train and test sets.

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [39]:
data = pd.read_csv("./data/cleaned_fake_job_postings.csv")
y = data.iloc[:, -1]
X = data.iloc[:, :-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

Before we move on to the next step, we will create new dataframes for the train and test set and save these as csv to data folder. We will use the following code to do the work.

```python 
# Resetting Index before we create new csv file for train and test
X_train.reset_index(inplace = True, drop = True)
y_train.reset_index(inplace = True, drop = True)
X_test.reset_index(inplace = True, drop = True)
y_test.reset_index(inplace = True, drop = True)

# Combining X and y (which is just fraudulent column) 
X_train["fraudulent"] = y_train
X_test["fraudulent"] = y_test

# Save the combined dataframe to csv
X_train.to_csv("./data/train_set.csv")
X_test.to_csv("./data/test_set.csv")
```

As we mentioned earlier, we want each word to be a feature of the dataset. To do this, we will use the **Bag of Words** method that will ensure each description in the dataset is represented only with the word count (e.g., if the description contains the word, the column will mark it as 1, otherwise 0). However, only using the Bag of Words is somewhat problematic. Since frequency is dependent on the length of the text, longer texts can calculate a higher frequency for some unnecessary words that rarely appear across the data as a whole. This clearly can cause big trouble when we come to feature selection. 

To remedy this problem, we will standardize the word frequency using **TF-IDF (Term Frequency-Inverse Document Frequency)** so that all frequencies can be weighted. It is a numerical statistic intended to reflect how important a word is to a document in a collection, and here is how we calculate it. 

$$ \text{TF-IDF} = \text{TF} \times \text{IDF} $$

where

$$TF = \frac{\text{number of times the word appears in the description}}{\text{total number of the word in the entire dataset}}$$

$$IDF = \log ( \frac{\text{number of description in the dataset}}{\text{number of description that contained the word}} )$$

Term Frequency (TF) measures how frequently a term occurs in a document. Inverse Document Frequency (IDF) is a factor that diminishes the weight of terms that occur very frequently in the document and increases the weight of words that occur rarely. As you observe here, as the word appears less frequently throughout the dataset, the IDF increases which decreases TF-IDF as a result. We gives more weight on the words that appear frequently across the entire dataset. This way, we can avoid possible outlier/confounding features in our dataset.

```{note}
In the original project, we had to extracting the features and converting those using TF-IDF were a seperate step. In Python, sklearn has very useful feature extraction encoder, "TfidfVectorizer", and this combines the extraction and conversion into a single step.
```

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Import train dataset
train_data = pd.read_csv("./data/train_set.csv")
train_data["description"].fillna('no_description', inplace = True)
vec = TfidfVectorizer(smooth_idf=True)
tfidf_description = vec.fit_transform(train_data["description"])

```{warning}
We have some NAs in the description column, which will cause the error with TfidfVectorizer if we don't impute it. I decided to replace all NAs with "no_description" because NA may indicate a strong signal for fraudulent. For instance, a fraudulent posting might have no job description.  
```

In [80]:
features_name = vec.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_description.toarray(), columns = features_name)
df_tfidfvect.head()

,aa,aaa,aaab,aab,aabc,aabd,aabf,aac,aaccd,aachen,...,zumero,zur,zurb,zurich,zusammenarbeitest,zusammenbringt,zweig,zyfax,zyka,zynga
0,0.164839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We will save our TfidfVectorizer as a pickle so that we can use it later to the test dataset.

```python
with open('./pickle/tfidfvec.pkl', 'wb') as f:
    pickle.dump(vec, f)
```
```{warning}
This step is essential because if we don't use the same encoder to train and test the set, it might raise the dimension error in any machine learning algorithm. For example, if we use a different encoder, the feature "planner" in the training set will not appear in the test set so that we can get the numbers of a feature. To avoid this error, we should apply the encoder that was trained by the training set to the test set. Sometimes, people combine train and test sets and run TfidfVectorizer to avoid this error, but we must always assume that we don't have any test set yet.
```